In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

In [2]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)


In [3]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


In [4]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))


In [5]:
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


210 documents
18 classes ['address', 'available_tables', 'book_table', 'contact', 'general', 'goodbye', 'greeting', 'hours', 'identity', 'menu', 'negative_feedback', 'offers', 'positive_feedback', 'recipe_enquiry', 'sanitation', 'suggest', 'veg_enquiry', 'vegan_enquiry']
253 unique lemmatized words [',', '.', 'a', 'able', 'about', 'address', 'ahead', 'all', 'also', 'am', 'amazing', 'an', 'and', 'any', 'are', 'available', 'awesome', 'bad', 'baked', 'behavior', 'best', 'book', 'bot', 'brownie', 'buy', 'bye', 'call', 'can', 'cant', 'capacity', 'care', 'changer', 'cheesy', 'choco', 'choose', 'clean', 'cleanliness', 'cold', 'concerned', 'contact', 'cool', 'could', 'covid', 'customer', 'cya', 'damn', 'day', 'dear', 'delicious', 'desk', 'detail', 'devoured', 'did', 'disappointed', 'discount', 'discovered', 'disgusting', 'dish', 'dissatisfied', 'do', 'dont', 'eat', 'ew', 'eww', 'executive', 'extremely', 'family', 'feedback', 'festive', 'find', 'fine', 'flavour', 'food', 'for', 'friendly', 'fuc

In [6]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))


In [7]:
# create our training data
training = []


In [8]:
# create an empty array for our output
output_empty = [0] * len(classes)


In [9]:
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
# output is a '0' for each tag and '1' for current tag (for each pattern)
        output_row = list(output_empty)
        output_row[classes.index(doc[1])] = 1
    
        training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


Training data created


<ipython-input-9-52e087aad6d4>:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [10]:
 # Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))



In [11]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")


C:\Users\akshata patil\anaconda3\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/200
10626/10626 [==============================] - 42s 4ms/step - loss: 0.1798 - accuracy: 0.9471
Epoch 2/200
10626/10626 [==============================] - 24s 2ms/step - loss: 0.0210 - accuracy: 0.9936
Epoch 3/200
10626/10626 [==============================] - 20s 2ms/step - loss: 0.0145 - accuracy: 0.9955
Epoch 4/200
10626/10626 [==============================] - 20s 2ms/step - loss: 0.0145 - accuracy: 0.9954
Epoch 5/200
10626/10626 [==============================] - 21s 2ms/step - loss: 0.0116 - accuracy: 0.9964 0s - loss: 0.0
Epoch 6/200
10626/10626 [==============================] - 20s 2ms/step - loss: 0.0088 - accuracy: 0.9971
Epoch 7/200
10626/10626 [==============================] - 20s 2ms/step - loss: 0.0081 - accuracy: 0.9970
Epoch 8/200
10626/10626 [==============================] - 20s 2ms/step - loss: 0.0073 - accuracy: 0.9977
Epoch 9/200
10626/10626 [==============================] - 20s 2ms/step - loss: 0.0084 - accuracy: 0.9972
Epoch 10/200
10626/10626 [=====

10626/10626 [==============================] - 23s 2ms/step - loss: 0.0021 - accuracy: 0.9993
Epoch 78/200
10626/10626 [==============================] - 38s 4ms/step - loss: 0.0026 - accuracy: 0.9992
Epoch 79/200
10626/10626 [==============================] - 46s 4ms/step - loss: 0.0031 - accuracy: 0.9993
Epoch 80/200
10626/10626 [==============================] - 43s 4ms/step - loss: 0.0027 - accuracy: 0.9992
Epoch 81/200
10626/10626 [==============================] - 45s 4ms/step - loss: 0.0022 - accuracy: 0.9994
Epoch 82/200
10626/10626 [==============================] - 45s 4ms/step - loss: 0.0025 - accuracy: 0.9991
Epoch 83/200
10626/10626 [==============================] - 47s 4ms/step - loss: 0.0023 - accuracy: 0.9993
Epoch 84/200
10626/10626 [==============================] - 41s 4ms/step - loss: 0.0022 - accuracy: 0.9993
Epoch 85/200
10626/10626 [==============================] - 44s 4ms/step - loss: 0.0029 - accuracy: 0.9993
Epoch 86/200
10626/10626 [========================

10626/10626 [==============================] - 38s 4ms/step - loss: 0.0014 - accuracy: 0.9996
Epoch 153/200
10626/10626 [==============================] - 38s 4ms/step - loss: 0.0021 - accuracy: 0.9995
Epoch 154/200
10626/10626 [==============================] - 39s 4ms/step - loss: 0.0014 - accuracy: 0.9996
Epoch 155/200
10626/10626 [==============================] - 38s 4ms/step - loss: 0.0015 - accuracy: 0.9997
Epoch 156/200
10626/10626 [==============================] - 39s 4ms/step - loss: 9.6055e-04 - accuracy: 0.9997
Epoch 157/200
10626/10626 [==============================] - 37s 3ms/step - loss: 9.5926e-04 - accuracy: 0.9997
Epoch 158/200
10626/10626 [==============================] - 39s 4ms/step - loss: 0.0012 - accuracy: 0.9996
Epoch 159/200
10626/10626 [==============================] - 37s 4ms/step - loss: 0.0011 - accuracy: 0.9996
Epoch 160/200
10626/10626 [==============================] - 39s 4ms/step - loss: 0.0012 - accuracy: 0.9996
Epoch 161/200
10626/10626 [=======